Before beginning task 3, make sure to run the following cell to import all necessary packages. If you need any additional packages, add the import statement(s) to the cell below and re-run the cell before adding and running code that uses the additional packages.


In [2]:
# Load all necessary packages
import numpy as np
import sklearn as skl
import six
import tensorflow as tf

# dataset
from aif360.datasets import AdultDataset

# metrics
from fklearn.metric_library import UnifiedMetricLibrary, classifier_quality_score

# models
from fklearn.scikit_learn_wrapper import LogisticRegression, KNeighborsClassifier, RandomForestClassifier, SVC
from aif360.algorithms.inprocessing import AdversarialDebiasing

# pre/post-processing algorithms
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification

from fklearn.fair_selection_aif import ModelSearch, DEFAULT_ADB_PARAMS

# Tutorial 3: fairkit-learn

Lastly, we show you how to train and evaluate models using fairkit-learn. You will use the knowledge from this tutorial to complete Task 3, so please read thoroughly and execute the code cells in order.

## Step 1: Import the dataset

First we need to import the dataset we will use for training and testing our model.

Below, we provide code that imports the Adult census dataset. **Note: a warning may pop up when you run this cell. As long as you don't see any errors in the code, it is fine to continue.**


In [3]:
data_orig = AdultDataset()

## Step 2: Set protected attributes

To use the grid search functionality provided by fairkit-learn, we again need to specify the privileged and unprivileged (protected) attributes. 

Below we provide code that stores the protected attributes (*race* is 0 for "Non-white", *sex* is 0 for "Female").

In [4]:
unprivileged = [{'race': 0, 'sex': 0}]
privileged = [{'race': 1, 'sex': 1}]

## Step 3: Specify parameters for grid search

Now we need to specify the various parameters required for the grid search provided by fairkit-learn. Each search parameter is a dictionary of options to include in the search. For each search parameter, you can input one or multiple options to consider. 

Below we provide code that sets parameters for a simple grid search across different hyper-parameter values for the Logistic Regression model, with and without the specified pre-/post-processing algorithms. We specify all performance and fairness metrics for the search -- given the way the classifier quality score is calculated, this cannot be added to the grid search and will be calculated later.

In [5]:
# we use one model here
models = {'LogisticRegression': LogisticRegression, 'RandomForestClassifier': RandomForestClassifier,
          'AdversarialDebiasing': AdversarialDebiasing}

# here we add all the metrics we want to evaluate on (performance and fairness)
metrics = {'UnifiedMetricLibrary': [UnifiedMetricLibrary,
                                    'accuracy_score',
                                    'average_odds_difference',
                                    'statistical_parity_difference',
                                    'equal_opportunity_difference',
                                    'disparate_impact'
                                   ]
          }

# Hyperparameters may either be specified as a dictionary of string to lists, or by an empty dictionary to
# use the default ones set by sklearn (or AIF360). The keys are the names of the hyperparameters, and the
# values and lists of possible values to form a grid search over (example shown with LogisticRegression)

# For the AdversarialDebiasing classifier, you would specify hyperparameters using the following dictionary
# entry:
# 'AdversarialDebiasing' : DEFAULT_ADB_PARAMS(unprivileged=unprivileged, privileged=privileged)

hyperparameters = {'LogisticRegression':{'penalty': ['l1', 'l2'], 'C': [0.1, 0.5, 1]},
                   'RandomForestClassifier':{},
                  'AdversarialDebiasing': DEFAULT_ADB_PARAMS(unprivileged=unprivileged, privileged=privileged)                  }

# this parameter is needed for the search and does not need to be modified
thresholds = [i * 10.0/100 for i in range(5)]

# Specify pre/post-processors as a list of initialized AIF360 pre/post-processing instances; 
# you can also run without any pre/post-processing algorithms (empty list)
processor_args = {'unprivileged_groups': unprivileged, 'privileged_groups': privileged}

# Options: DisparateImpactRemover(), Reweighing(unprivileged_groups=unprivileged,privileged_groups=privileged), or both
preprocessors=[DisparateImpactRemover(), Reweighing(**processor_args)]
# Options: CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged,privileged_groups=privileged), RejectOptionClassification(unprivileged_groups=unprivileged,privileged_groups=privileged), or both
postprocessors=[CalibratedEqOddsPostprocessing(**processor_args), RejectOptionClassification(**processor_args)]


## Step 4: Run the grid search

Now that we've set all the parameters necessary for the grid search, we're ready to run it. The output of the grid search is saved to a .csv file.

Below we provide code that creates and uses the `ModelSearch` object to run a grid search over the parameters we specified and saves the output to a .csv file in the specified directory.  **The search take a while to complete. Wait until the search completes before attempting to execute more cells.**

**Note: warnings may appear during search, however, as long as you don’t see any code errors it is fine to continue.** 


In [ ]:
Search = ModelSearch(models, metrics, hyperparameters, thresholds)
Search.grid_search(data_orig, privileged=privileged, unprivileged=unprivileged, preprocessors=preprocessors, postprocessors=postprocessors)

Search.to_csv("fklearn/interface/static/data/test-file.csv")

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

...

epoch 2; iter: 200; batch classifier loss: 11.996395; batch adversarial loss: 0.580438
epoch 3; iter: 0; batch classifier loss: 1.880728; batch adversarial loss: 0.550961
epoch 3; iter: 200; batch classifier loss: 1.629515; batch adversarial loss: 0.433547
epoch 4; iter: 0; batch classifier loss: 5.742574; batch adversarial loss: 0.511901
epoch 4; iter: 200; batch classifier loss: 1.285036; batch adversarial loss: 0.428534
epoch 5; iter: 0; batch classifier loss: 1.638271; batch adversarial loss: 0.500204
epoch 5; iter: 200; batch classifier loss: 3.124608; batch adversarial loss: 0.427560
epoch 6; iter: 0; batch classifier loss: 1.472153; batch adversarial loss: 0.480381
epoch 6; iter: 200; batch classifier loss: 3.251405; batch adversarial loss: 0.416129
epoch 7; iter: 0; batch classifier loss: 0.951004; batch adversarial loss: 0.528275
epoch 7; iter: 200; batch classifier loss: 0.420186; batch adversarial loss: 0.345917
epoch 8; iter: 0; batch classifier loss: 0.416968; batch advers

epoch 0; iter: 200; batch classifier loss: 6.966563; batch adversarial loss: 0.619411
epoch 1; iter: 0; batch classifier loss: 11.899750; batch adversarial loss: 0.596918
epoch 1; iter: 200; batch classifier loss: 3.605996; batch adversarial loss: 0.519822
epoch 2; iter: 0; batch classifier loss: 2.431391; batch adversarial loss: 0.558654
epoch 2; iter: 200; batch classifier loss: 1.908173; batch adversarial loss: 0.536450
epoch 3; iter: 0; batch classifier loss: 2.704957; batch adversarial loss: 0.428824
epoch 3; iter: 200; batch classifier loss: 2.023658; batch adversarial loss: 0.423146
epoch 4; iter: 0; batch classifier loss: 0.612531; batch adversarial loss: 0.457823
epoch 4; iter: 200; batch classifier loss: 1.992843; batch adversarial loss: 0.404866
epoch 5; iter: 0; batch classifier loss: 2.235471; batch adversarial loss: 0.346411
epoch 5; iter: 200; batch classifier loss: 2.007464; batch adversarial loss: 0.377987
epoch 6; iter: 0; batch classifier loss: 2.075370; batch advers

epoch 48; iter: 200; batch classifier loss: 0.339236; batch adversarial loss: 0.489796
epoch 49; iter: 0; batch classifier loss: 0.313936; batch adversarial loss: 0.403936
epoch 49; iter: 200; batch classifier loss: 0.301604; batch adversarial loss: 0.380497
epoch 0; iter: 0; batch classifier loss: 5.011238; batch adversarial loss: 0.769551
epoch 0; iter: 200; batch classifier loss: 7.425304; batch adversarial loss: 0.673340
epoch 1; iter: 0; batch classifier loss: 10.360415; batch adversarial loss: 0.639864
epoch 1; iter: 200; batch classifier loss: 5.809455; batch adversarial loss: 0.553599
epoch 2; iter: 0; batch classifier loss: 2.536075; batch adversarial loss: 0.569865
epoch 2; iter: 200; batch classifier loss: 2.650975; batch adversarial loss: 0.489792
epoch 3; iter: 0; batch classifier loss: 2.529573; batch adversarial loss: 0.528549
epoch 3; iter: 200; batch classifier loss: 1.404363; batch adversarial loss: 0.468081
epoch 4; iter: 0; batch classifier loss: 1.008184; batch adv

epoch 46; iter: 200; batch classifier loss: 0.287770; batch adversarial loss: 0.455077
epoch 47; iter: 0; batch classifier loss: 0.354398; batch adversarial loss: 0.548461
epoch 47; iter: 200; batch classifier loss: 0.360498; batch adversarial loss: 0.385005
epoch 48; iter: 0; batch classifier loss: 0.323066; batch adversarial loss: 0.389321
epoch 48; iter: 200; batch classifier loss: 0.234073; batch adversarial loss: 0.348227
epoch 49; iter: 0; batch classifier loss: 0.428525; batch adversarial loss: 0.376637
epoch 49; iter: 200; batch classifier loss: 0.403683; batch adversarial loss: 0.389120
epoch 0; iter: 0; batch classifier loss: 34.946327; batch adversarial loss: 1.059033
epoch 0; iter: 200; batch classifier loss: 7.362378; batch adversarial loss: 0.867830
epoch 1; iter: 0; batch classifier loss: 1.984664; batch adversarial loss: 0.686881
epoch 1; iter: 200; batch classifier loss: 4.448523; batch adversarial loss: 0.683302
epoch 2; iter: 0; batch classifier loss: 6.946095; batch

epoch 44; iter: 200; batch classifier loss: 0.372456; batch adversarial loss: 0.367216
epoch 45; iter: 0; batch classifier loss: 0.347583; batch adversarial loss: 0.461049
epoch 45; iter: 200; batch classifier loss: 0.367362; batch adversarial loss: 0.332134
epoch 46; iter: 0; batch classifier loss: 0.382968; batch adversarial loss: 0.481443
epoch 46; iter: 200; batch classifier loss: 0.349227; batch adversarial loss: 0.436819
epoch 47; iter: 0; batch classifier loss: 0.362101; batch adversarial loss: 0.349596
epoch 47; iter: 200; batch classifier loss: 0.393379; batch adversarial loss: 0.505211
epoch 48; iter: 0; batch classifier loss: 0.344760; batch adversarial loss: 0.385158
epoch 48; iter: 200; batch classifier loss: 0.294458; batch adversarial loss: 0.358625
epoch 49; iter: 0; batch classifier loss: 0.360043; batch adversarial loss: 0.362116
epoch 49; iter: 200; batch classifier loss: 0.358058; batch adversarial loss: 0.427663
epoch 0; iter: 0; batch classifier loss: 26.145233; b

## Step 5: Render visualization of search results

Along with the ability to run a grid search, fairkit-learn also provides functionality to visualize the results of the grid search. Fairkit-learn uses Bokeh to render a visualization within the notebook, which you can use when completing the next task to explore trained models' performance and fairness.

The visualzation includes a graph that plots the search results that are pareto optimal. Each data point in the graph is a model with its own settings (e.g., hyper-parameters, pre/post processing). Each model class has its own color to make it easier to see which models are being shown in the visualization. To get more information on each model's settings, hover over the data point of interest; a tooltip will pop up with model settings.

Within the visualization, you can control what metrics and models are being included in the visualization. The drop down menus allow you to specify the x and y axes for the graph. The checklist below the list of models allows you to select which metrics can be considered in the graph.

To view the *Pareto frontier* for any two metrics (e.g., accuracy and disparate impact), select those two metrics from the drop down menu and **only** check those boxes in the checklist.

Below we provide code to load Bokeh and plot the results from the search in the interactive plot.

In [ ]:
# Import packages for visualization
from bokeh.io import output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# load Bokeh
output_notebook()

In [ ]:
from fklearn.interface.plot import *

# Define function that takes in a document and attaches the bokeh server to it
def modify_doc(doc):
    
    # Load custom styles (for notebook only)
    custom_css = Div(text="<link rel='stylesheet' type='text/css' href='fklearn/interface/static/css/styles-notebook.css'>")
    add_btn = Button(label="Add Plot", button_type="success")
    remove_btn = Button(label="Remove Plot", button_type="danger")

    # Construct our viewport
    l = layout([
        [custom_css],
        create_plot("fklearn/interface/static/data/test-file.csv")
    ], sizing_mode="fixed", css_classes=["layout-container"])

    doc.add_root(l)
    
# Set up the application
handler = FunctionHandler(modify_doc)
app = Application(handler)

# Render visualization in the notebook
show(app)

## Step 6: Export visualization (optional)

The visualization can be viewed within this notebook and re-rendered as many times as needed, but can also be exported for future viewing and comparison to other plots. You can export the visualization and relevant information by clicking the ``Export Plot`` button in the visualization. 

This will save create two files: plot.png and plot.json.
Plot.png is an image of the plot.
Plot.json is a JSON file with the informational bits from the plot, such as what models are being shown and what metrics are selected.

Each time the export button is clicked, if plot.png and plot.json exist they are overwritten. If you wish to save plots for comparision, make sure you rename each file after export.

## Step 7: Evaluate overall model quality

Now that we've explored the various model configurations and their performance and fairness, we are ready to select the model(s) that we want to evaluate for overall quality.

To do so, you will need to create and train the model(s) (with proper hyperparameters, pre-processing, and post-processing as specified in the search output) you selected and then evaluate overall model quality.

Below we provide (commented) code that shows how to intialize the various models and algorithms you have access to. You can use the code provided or modify as you see fit when completing the task.

In [ ]:
# split dataset for evaluation
# data_orig_train, data_orig_test = data_orig.split([0.7], shuffle=False)

# model is populated with default values; modifying parameters is allowed but optional
# model = LogisticRegression(penalty='l2', dual=False,tol=0.0001,C=1.0,
#                       fit_intercept=True,intercept_scaling=1,class_weight=None,
#                       random_state=None,solver='liblinear',max_iter=100, 
#                       multi_class='warn',verbose=0,warm_start=False,
#                       n_jobs=None)

#model = KNeighborsClassifier(n_neighbors=5,weights='uniform',algorithm='auto',
#                          leaf_size=30,p=2,metric='minkowski',metric_params=None,
#                          n_jobs=None)

#model = RandomForestClassifier(n_estimators='warn',criterion='gini',max_depth=None,
#                            min_samples_leaf=1,min_weight_fraction_leaf=0.0,
#                            min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, 
#                             random_state=None, verbose=0, warm_start=False, class_weight=None)

#model = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto_deprecated', coef0=0.0, shrinking=True, 
#          probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
#          max_iter=-1, decision_function_shape='ovr', random_state=None)

# If this is not your first time creating the Adversarial Debiasing model, to avoid future errors,
# uncomment the code below before running the code that initializing TensorFlow session and model:
# sess.close()
# tf.reset_default_graph()

#sess = tf.Session()
#model = AdversarialDebiasing(privileged_groups=privileged,
#                          unprivileged_groups=unprivileged,
#                          scope_name='debiased_classifier',
#                          debias=True,
#                          sess=sess)


# you can modify repair level (optional)
#pre_alg = DisparateImpactRemover(repair_level=1.0)
# training data
#pre_train_data = pre_alg.fit_transform(data_orig_train)
# test data
#pre_test_data = pre_alg.fit_transform(data_orig_test)



# Reweighing

#pre_alg = Reweighing(unprivileged_groups=unprivileged, privileged_groups=privileged)
# train
#pre_alg = pre_alg.fit(dataset_orig_train)
#pre_train_data = pre_alg.transform(data_orig_train)
# test
#pre_alg = pre_alg.fit(dataset_orig_test)
#pre_test_data = pre_alg.transform(data_orig_test)


# train model with pre-processed data
#model.fit(pre_train_data)

# train model with original data 
# model.fit(data_orig_train)


# process trained model
# Calibrated Equal Odds
#post_alg = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged,
#                                         privileged_groups=privileged,
#                                         cost_constraint='weighted',
#                                         seed=None)



# Reject Option Classification 
# With this algorithm, you can specify "metric_name" with the metric you want to optimize for.
# The options are "Statistical parity difference", "Average odds difference", or "Equal opportunity difference"

# post_alg = RejectOptionClassification(unprivileged_groups=unprivileged,
#                                      privileged_groups=privileged,
#                                      low_class_thresh=0.01,
#                                      high_class_thresh=0.99,num_class_thresh=100, 
#                                      num_ROC_margin=50,metric_name='Statistical parity difference',
#                                      metric_ub=0.05, metric_lb=-0.05)



# test with pre-processed test data
#predictions = model.predict(pre_test_data)

# test with original test data
# predictions = model.predict(data_orig_test)


# fit with post-processing model using pre-processed data
#post_model = post_alg.fit(pre_test_data, predictions)

# fit with post-processing model using original data
# post_model = post_alg.fit(data_orig_test, predictions)



# update predictions using post-processed model
#predictions = post_model.predict(pre_test_data)

# evaluate overall model quality on post-processed model
#quality_score = classifier_quality_score(post_model, predictions, 
#                                             unprivileged_groups=unprivileged, 
#                                             privileged_groups=privileged)

# evaluate overall model quality on model without post-processing
#quality_score = classifier_quality_score(model, predictions, 
#                                             unprivileged_groups=unprivileged, 
#                                             privileged_groups=privileged)

#print("Overall quality = " + str(quality_score))

# Task 3: Model evaluation with fairkit-learn

Your turn again! Use what you learned in the above tutorial to train and evaluate models for performance, fairness, and overall quality. You will use functionality provided by fairkit-learn to meet the following goals:

1. **Describe a model you believe will perform the best (e.g., have the highest accuracy score).** 

2. **Describe a model you believe will be the most fair, regardless of performance (e.g., minimizes the value of difference fairness metrics or maximizes disparate impact).** 

3. **Describe a model you believe will best balance both performance and fairness (e.g., have the highest classifier quality score).** 

Make sure you include any modifications to model hyper-parameters and any pre-/post-processing algorithms used. **As a reminder, there is no "absolute best" model for each of the above goals. You are expected to explore the space of model configurations available to find a model that best meets the above goals.**

**Keep in mind, training machine learning models is often a time intensive endeavor.** One way you can minimize time to finish this task is to minimize the search space (e.g., number of models included in a single search). You can also minimize time when evaluating the number of times you have to, for example, train a given model to then evaluate it. You can do this by putting the code that initializes and trains your model(s) in its own separate cell and only execute this cell when needed.

## Submitting your response

Once you feel you've met the above goals, go to the Evaluating ML Models Exercise Response Form to enter your responses under the section labeled 'Task 3'.

If you accidentally closed your response form, check your email for the link to re-open it.

In [ ]:
# TODO : Use this cell to write code for completing task 3



Once you've completed this final task, make sure you're satisfied with your responses, complete the exercise feedback portion, and submit the form.